In [97]:
import pandas as pd
import requests
import re

In [98]:
# creating DF from the data from the other API

waqi_df = pd.read_csv('waqi_df.csv')
waqi_df

,Unnamed: 0,cities,aqi,latitude,longitude,pm25,pm10,o3,uvi,date
0,0,Tokyo,13,35.641463,139.698171,1.0,21.0,1.0,0.0,11-13-2021
1,1,Jakarta,142,-6.182536,106.834236,42.0,173.0,7.0,3.0,11-13-2021
2,2,New York,103,40.712784,-74.005941,97.0,27.0,8.0,1.0,11-13-2021
3,3,Seoul,38,37.566535,126.977969,74.0,24.0,3.0,0.0,11-13-2021
4,4,Manila,-,14.599512,120.984220,NaN,NaN,NaN,NaN,11-13-2021
...,...,...,...,...,...,...,...,...,...,...
596,992,At-Ta'If,27,21.271944,40.454722,NaN,88.0,42.0,2.0,11-13-2021
597,993,Dali,34,25.581100,100.217100,42.0,19.0,4.0,0.0,11-13-2021
598,995,Jinzhou,173,41.095120,121.127004,160.0,13.0,6.0,1.0,11-13-2021
599,999,Duisburg,26,51.437686,6.771867,NaN,32.0,7.0,0.0,11-13-2021


In [99]:
cities = waqi_df['cities'].to_list()


In [100]:
# temporary list... using for testing. will be using the cities column from the waqi_df in the future

#cities = ['Tokyo', 'Jakarta', 'New York', 'Seoul', 'Manila']

In [101]:
# creating a new df

spott_df = pd.DataFrame(cities, columns=['Cities'])

In [102]:
spott_df

,Cities
0,Tokyo
1,Jakarta
2,New York
3,Seoul
4,Manila
...,...
596,At-Ta'If
597,Dali
598,Jinzhou
599,Duisburg


In [103]:
url = "https://spott.p.rapidapi.com/places"

In [104]:
api_key = "34147b430dmshfc2dec28a0e0c7cp1c0309jsn7f6433113e10" #under the aelutz2000 account

In [105]:
# returns the top search information from the Spott API for each city

def get_city_data(city):
    querystring = {"language": "en","q":city}

    headers = {
        'x-rapidapi-host': "spott.p.rapidapi.com",
        'x-rapidapi-key': api_key
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    try:
        return response.json()[0]
    except Exception as e:
        #print(e)
        return "something wrong with request {}".format(e)

In [106]:
# dictionary collects ll the API data
info_dict = {}

In [107]:
# adds each city's data to the big dictionary 
for city in cities:
    info_dict[city] = get_city_data(city)

In [108]:
info_dict

{'Tokyo': {'id': '1850147',
  'geonameId': 1850147,
  'type': 'CITY',
  'name': 'Tokyo',
  'population': 8336599,
  'elevation': 44,
  'timezoneId': 'Asia/Tokyo',
  'country': {'id': 'JP',
   'geonameId': 1861060,
   'name': 'Japan',
   'localizedName': 'Japan'},
  'adminDivision1': {'id': 'JP.40',
   'geonameId': 1850144,
   'name': 'Tokyo',
   'localizedName': 'Tokyo'},
  'score': 121.36106,
  'localizedName': 'Tokyo',
  'coordinates': {'latitude': 35.6895, 'longitude': 139.692}},
 'Jakarta': {'id': '1642911',
  'geonameId': 1642911,
  'type': 'CITY',
  'name': 'Jakarta',
  'population': 8540121,
  'elevation': 16,
  'timezoneId': 'Asia/Jakarta',
  'country': {'id': 'ID',
   'geonameId': 1643084,
   'name': 'Indonesia',
   'localizedName': 'Indonesia'},
  'adminDivision1': {'id': 'ID.04',
   'geonameId': 1642907,
   'name': 'Jakarta Special Capital Region',
   'localizedName': 'Jakarta Special Capital Region'},
  'score': 202.2663,
  'localizedName': 'Jakarta',
  'coordinates': {'lat

In [109]:
# returns population info, references data from the big city info dictionary 

def add_population(name):
    city_info = info_dict[name]
    if city_info == 'something wrong with request list index out of range':
        return None
    else:
        return city_info['population']


In [110]:
spott_df['Population'] = spott_df['Cities'].apply(add_population)

In [111]:
spott_df.head()

,Cities,Population
0,Tokyo,8336599.0
1,Jakarta,8540121.0
2,New York,8175133.0
3,Seoul,10349312.0
4,Manila,1600000.0


In [112]:
# returns elevation info, references data from the big city info dictionary 

def add_elevation(name):
    city_info = info_dict[name]
    if city_info == 'something wrong with request list index out of range':
        return None
    else:
        return city_info['elevation']


In [113]:
spott_df['Elevation'] = spott_df['Cities'].apply(add_elevation)

In [114]:
spott_df.head()

,Cities,Population,Elevation
0,Tokyo,8336599.0,44.0
1,Jakarta,8540121.0,16.0
2,New York,8175133.0,10.0
3,Seoul,10349312.0,38.0
4,Manila,1600000.0,13.0


In [115]:
spott_df

,Cities,Population,Elevation
0,Tokyo,8336599.0,44.0
1,Jakarta,8540121.0,16.0
2,New York,8175133.0,10.0
3,Seoul,10349312.0,38.0
4,Manila,1600000.0,13.0
...,...,...,...
596,At-Ta'If,530848.0,1672.0
597,Dali,14760.0,1517.0
598,Jinzhou,604269.0,26.0
599,Duisburg,504358.0,38.0


In [119]:
spott_df.to_csv('spott_df.csv')